In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [2]:
import pandas as pd
from transformers import (set_seed, AutoConfig, AutoModelForCausalLM,
                          AutoTokenizer, 
                            BitsAndBytesConfig)
import torch
from datasets import load_dataset
from tqdm import tqdm
from sklearn.metrics import recall_score, precision_score, f1_score
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [3]:
device = "cuda"
model_name = "ISTA-DASLab/Meta-Llama-3-8B-Instruct"

In [4]:
train = load_dataset("brighter-dataset/BRIGHTER-emotion-categories", "rus", split="train")

In [5]:
emotion_cols = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
emotion_map = {
    'anger': 'гнев',
    'disgust': 'отвращение', 
    'fear': 'страх',
    'joy': 'радость',
    'sadness': 'грусть',
    'surprise': 'удивление'
}

In [6]:
def create_labels(examples):
    labels = []
    for i in range(len(examples['text'])):
        label = [examples[col][i] for col in emotion_cols]
        labels.append(label)
    examples['labels'] = labels
    return examples

train = train.map(create_labels, batched=True)

In [7]:
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

print("Создание эмбеддингов для тренировочных данных...")
train_texts = [row['text'] for row in train]
train_embeddings = embedding_model.encode(train_texts, show_progress_bar=True)

Создание эмбеддингов для тренировочных данных...


Batches:   0%|          | 0/84 [00:00<?, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_cache=False,
    trust_remote_code=True,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
tokenizer.pad_token = tokenizer.eos_token

In [10]:
test = load_dataset("brighter-dataset/BRIGHTER-emotion-categories", "rus", split="test")

In [11]:
seed = 42
set_seed(seed)

In [12]:
def find_similar_examples(query_text, train_data, train_embeddings, k=3):
    """
    Находит k наиболее похожих примеров из трейна для данной эмоции
    """
    query_embedding = embedding_model.encode([query_text])
    
    similarities = cosine_similarity(query_embedding, train_embeddings)[0]
    
    similar_indices = np.argsort(similarities)[::-1]
    
    examples = []
    for idx in similar_indices:
        if len(examples) >= k:
            break
        example = train_data[int(idx)]
        emotions = []
        for col in emotion_cols:
            if example[col] == 1:
                emotions.append(emotion_map[col])
        examples.append((example['text'], emotions))
    
    return examples[:k]

def create_few_shot_prompt(query_text, examples):
    """
    Создает few-shot промпт с примерами
    """
    prompt = f"""Ты эксперт по анализу эмоций в тексте. 
    Определи, какие эмоции выражены в тексте из списка [гнев, отвращение, страх, радость, грусть, удивление.]
    Эмоций может быть несколько, а может и вовсе не быть. Формат вывода: только названия эмоций. Если нет эмоций, то оставь пустой список.

    ВАЖНЫЕ ПРАВИЛА:
    - Ставь 1 ТОЛЬКО если эмоция выражена ЯВНО через конкретные слова, фразы или контекст
    - НЕ додумывай скрытые эмоции - только то, что написано прямо
    - При сомнениях выбирай 0
    
    Примеры:
    """
    
    # Добавляем примеры
    for i, (example_text, emotions) in enumerate(examples, 1):
        prompt += f"Текст: {example_text}\n"
        prompt += f"Ответ: {emotions}\n\n"
    
    # Добавляем целевой текст
    prompt += f"Проанализируй этот текст по тем же критериям:\n"
    prompt += f"Текст: {query_text}\n"
    prompt += f"Ответ:"
    
    return prompt


In [13]:
responses = []
prompts = []

for row in tqdm(test):
    labels = []
    similar_examples = find_similar_examples(
        row['text'], 
        train, 
        train_embeddings, 
        k=2
    )
    
    prompt = create_few_shot_prompt(
        row['text'], 
        similar_examples
    )
    
    prompts.append(prompt)
    
    messages = [{"role": "user", "content": prompt}]
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    
    input_ids = tokenizer(formatted_prompt, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(
        input_ids=input_ids,
        temperature=0.1,
        do_sample=True,
        top_k=1,
        top_p=0.9,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
    generated_ids = outputs[0][input_ids.shape[1]:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    responses.append(response)

100%|███████████████████████████████████████| 2000/2000 [21:50<00:00,  1.53it/s]


In [14]:
true_emotions = []

In [15]:
for i, row in tqdm(enumerate(test)):
    true_emotion = [row[col] for col in emotion_cols]
    true_emotions.append(true_emotion)

2000it [00:00, 12233.75it/s]


In [16]:
responses[:10]

["assistant\n\n['отвращение', 'страх']",
 'assistant\n\n[радость]',
 "assistant\n\n['отвращение', 'гнев']",
 "assistant\n\n['отвращение', 'гнев']",
 "assistant\n\n['радость']",
 'assistant\n\n[]',
 "assistant\n\n['гнев', 'отвращение']",
 'assistant\n\n[]',
 'assistant\n\n[гнев, радость]',
 "assistant\n\n['страх']"]

In [17]:
emotion_values = list(emotion_map.values())

def parse_response_to_binary(response: str) -> list[int]:
    response = response.lower()
    return [1 if emo in response else 0 for emo in emotion_values]

In [18]:
pred_emotions = [parse_response_to_binary(response) for response in responses]

In [20]:
for average in ['micro', 'macro']:
    recall = recall_score(true_emotions, pred_emotions, average=average, zero_division=0)
    precision = precision_score(true_emotions, pred_emotions, average=average, zero_division=0)
    f1 = f1_score(true_emotions, pred_emotions, average=average, zero_division=0)
    print(f'{average.upper()} recall: {recall:.4f}, precision: {precision:.4f}, f1: {f1:.4f}')

MICRO recall: 0.7919, precision: 0.4753, f1: 0.5941
MACRO recall: 0.7786, precision: 0.5473, f1: 0.6047


In [21]:
class_recall = recall_score(true_emotions, pred_emotions, average=None, zero_division=0)
class_precision = precision_score(true_emotions, pred_emotions, average=None, zero_division=0)
class_f1 = f1_score(true_emotions, pred_emotions, average=None, zero_division=0)

for i, (eng_emotion, rus_emotion) in enumerate(emotion_map.items()):
    print(f'{rus_emotion}: recall: {class_recall[i]:.4f}, precision: {class_precision[i]:.4f}, f1: {class_f1[i]:.4f}')

гнев: recall: 0.8894, precision: 0.5560, f1: 0.6843
отвращение: recall: 0.9672, precision: 0.2622, f1: 0.4126
страх: recall: 0.9167, precision: 0.6851, f1: 0.7842
радость: recall: 0.8756, precision: 0.5121, f1: 0.6463
грусть: recall: 0.5674, precision: 0.4908, f1: 0.5263
удивление: recall: 0.4553, precision: 0.7778, f1: 0.5744


In [ ]:
def find_similar_examples(query_text, train_data, train_embeddings, emotion_col, k=4):
    """
    Находит k наиболее похожих примеров из трейна для данной эмоции
    """
    query_embedding = embedding_model.encode([query_text])
    
    similarities = cosine_similarity(query_embedding, train_embeddings)[0]
    
    similar_indices = np.argsort(similarities)[::-1]
    
    positive_examples = []
    negative_examples = []
    
    for idx in similar_indices:
        if len(positive_examples) >= k//2 and len(negative_examples) >= k//2:
            break
            
        example = train_data[int(idx)]
        
        if example[emotion_col] == 1 and len(positive_examples) < k//2:
            positive_examples.append((example['text'], 1))
        elif example[emotion_col] == 0 and len(negative_examples) < k//2:
            negative_examples.append((example['text'], 0))
    
    all_examples = positive_examples + negative_examples
    
    return all_examples[:k]

def create_few_shot_prompt(query_text, emotion_name, examples):
    """
    Создает few-shot промпт с примерами и строгими критериями
    """
    prompt = f"""Ты эксперт по анализу эмоций в тексте. Определи, ЯВНО ли выражена эмоция "{emotion_name}" в тексте.

ВАЖНЫЕ ПРАВИЛА:
- Ставь 1 ТОЛЬКО если эмоция выражена ЯВНО через конкретные слова, фразы или контекст
- Ставь 0 если эмоция НЕ выражена явно, даже если можно предположить её наличие
- НЕ додумывай скрытые эмоции - только то, что написано прямо
- При сомнениях выбирай 0

Примеры:
"""
    
    # Добавляем примеры
    for i, (example_text, label) in enumerate(examples, 1):
        prompt += f"Текст: {example_text}\n"
        prompt += f"Ответ: {label}\n\n"
    
    # Добавляем целевой текст
    prompt += f"Проанализируй этот текст по тем же критериям:\n"
    prompt += f"Текст: {query_text}\n"
    prompt += f"Ответ:"
    
    return prompt

In [ ]:
responses = []
prompts = []

for row in tqdm(test):
    labels = []
    for col in emotion_cols:
        similar_examples = find_similar_examples(
            row['text'], 
            col,
            train, 
            train_embeddings, 
            k=2
        )
        
        prompt = create_few_shot_prompt(
            row['text'], 
            similar_examples
        )
        
        prompts.append(prompt)
        
        messages = [{"role": "user", "content": prompt}]
        formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
        
        input_ids = tokenizer(formatted_prompt, return_tensors="pt").input_ids.to(device)
        outputs = model.generate(
            input_ids=input_ids,
            temperature=0.1,
            do_sample=True,
            top_k=1,
            top_p=0.9,
            max_new_tokens=2,
            pad_token_id=tokenizer.eos_token_id
        )
    
        generated_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
        generated_ids = outputs[0][input_ids.shape[1]:]
        response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
        labels.append(response)
    responses.append(labels)